In [1]:
cd ../..

/Users/tom/Documents/REPO/guitar-set


In [2]:
import guitar_set.pitch_tracker as ptk
import guitar_set.interpreter as itp
import librosa
import jams
import mir_eval.display
import mir_eval.sonify
import librosa.display
import numpy as np
import os
import numba
import scipy.signal as signal
import matplotlib.pyplot as plt
from IPython.display import Audio

def compute_novelty_spec_flux(y, fs, hop_size=512):
    """ Compute novelty function using spectural flux.
    Parameters
    ----------
    y : ndarray
    (T,) mono time domian signal
    fs : int
    sample rate of y
    win_size : int
    hop_size : int

    Returns
    -------
    novelty : ndarray
    times : ndarray
    fs_novelty : float
    """
    S = librosa.feature.melspectrogram(y=y, sr=fs, n_mels=256,
                                       hop_length=hop_size)
    S_dB = librosa.power_to_db(S, ref=np.max)
    dS_dt = np.diff(S, axis=1)
    # HW Rectify dS_dt
    for x in np.nditer(dS_dt, op_flags=['readwrite']):
        x[...] = max(x, 0)
    #

    weight_vec = np.append(np.zeros(128), np.ones(128))
    weight_vec += 1

    weighted_sf = np.dot(np.diag(weight_vec), dS_dt)

    # output gathering
    novelty = np.mean(weighted_sf, axis=0)
    fs_novelty = float(fs) / hop_size
    times = np.array([float(i) / fs_novelty for i in range(len(novelty))])

    return novelty, times, fs_novelty


def medfilt(x, k):
    """Apply a length-k median filter to a 1D array x.
    Boundaries are extended by repeating endpoints.
    """
    assert k % 2 == 1, "Median filter length must be odd."
    assert x.ndim == 1, "Input must be one-dimensional."
    k2 = (k - 1) // 2
    y = np.zeros((len(x), k), dtype=x.dtype)
    y[:, k2] = x
    for i in range(k2):
        j = k2 - i
        y[j:, i] = x[:-j]
        y[:j, i] = x[0]
        y[:-j, -(i + 1)] = x[j:]
        y[-j:, -(i + 1)] = x[-1]
    return np.median(y, axis=1)


def onsets_from_novelty(novelty, times, fs, w_c=3, medfilt_len=11,
                        offset=0.001):
    # smoothing using butterworth and normalize
    nyq = fs / 2.0
    B, A = signal.butter(1, w_c / nyq, 'low')
    novelty_smoothed = signal.filtfilt(B, A, novelty)
    novelty_smoothed = novelty_smoothed / np.max(np.abs(novelty_smoothed))

    # adaptive thresholding
    thresh = medfilt(novelty_smoothed, medfilt_len) + offset

    # onset detection
    peak_idx = librosa.util.peak_pick(novelty_smoothed, 3, 3, 3, 5, 5e-5, 10)

    pruned_peak_idx = []
    for p_idx in peak_idx:
        if novelty_smoothed[p_idx] > thresh[p_idx]:  # it made it
            pruned_peak_idx.append(p_idx)
    return pruned_peak_idx


def load_dirpath(mono_dir_path):
    """:returns
        ys, sr
    """
    ys = []
    for string_number in range(6):
        mono_path = os.path.join(mono_dir_path, '{}.wav'.format(string_number))

        y, sr = librosa.load(mono_path)
        print('length of y: {}, sr: {}'.format(len(y), sr))
        ys.append(y)
    return np.asarray(ys), sr


def qxi_onset(y, sr):
    novelty, times, fs_novelty = compute_novelty_spec_flux(y, sr, hop_size=512)
    pruned_peak_idx = onsets_from_novelty(novelty, times, fs_novelty)
    return times[pruned_peak_idx]

In [3]:
ys, sr = load_dirpath('/Users/tom/Music/DataSet/test-set_processed/eh_BN1-129-Eb_c_hex_cln/')

length of y: 485100, sr: 22050
length of y: 485100, sr: 22050
length of y: 485100, sr: 22050
length of y: 485100, sr: 22050
length of y: 485100, sr: 22050
length of y: 485100, sr: 22050


In [24]:
ys_sec = ys[:, 0:80000]

In [25]:
Audio(data=ys_sec[0], rate=sr)

In [26]:
Audio(data=ys_sec[1], rate=sr)

In [27]:
Audio(data=ys_sec[2], rate=sr)

In [28]:
Audio(data=ys_sec[3], rate=sr)

In [29]:
Audio(data=ys_sec[4], rate=sr)

In [30]:
Audio(data=ys_sec[5], rate=sr)

In [32]:
Audio(data=ys_sec[1] + ys_sec[2] + ys_sec[4], 
      rate=sr)